In [1]:
import cv2
import csv
import datetime
import numpy as np
from deepface import DeepFace

# Définir les catégories d'âge
age_categories = {
    'children': (0, 12),
    'teenagers': (13, 19),
    'young_adults': (20, 35),
    'middle_adults': (36, 50),
    'elderly': (51, np.inf)
}

# Fonction pour attribuer une catégorie d'âge
def categorize_age(age):
    for category, (lower, upper) in age_categories.items():
        if lower <= age <= upper:
            return category
    return 'unknown'  # Si l'âge ne correspond à aucune catégorie

def detect_emotion_from_stream():
    DeepFace.model = 'VGG-Face'
    
    # Capture vidéo à partir de la webcam (index 0)
    cap = cv2.VideoCapture(0)
    
    # Ouvrir un fichier CSV en mode écriture
    csv_file = open('C:/Users/lenovo/Desktop/tesst/deepface/facial_information/emotions.csv', 'w', newline='', encoding='utf-8')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['ID', 'Date', 'Time', 'Age', 'Age Category', 'Gender', 'Emotion', 'Frequency'])
    
    # Dictionnaire pour stocker les informations des visages détectés
    face_dict = {}
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convertir l'image OpenCV en format requis par DeepFace (BGR -> RGB)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Convertir l'image en niveaux de gris pour la détection de visage
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Charger le détecteur de visage Haarcascade
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
        # Détecter les visages dans l'image
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        # Dessiner des cadres autour des visages détectés
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Analyser l'image avec DeepFace (enforce_detection=False pour éviter la détection de visage)
        emotions = DeepFace.analyze(rgb_frame, actions=['age', 'gender', 'emotion'], enforce_detection=False)
        
        # Récupérer les informations nécessaires à partir de la liste
        age = emotions[0]['age']
        dominant_emotion = emotions[0]['dominant_emotion']
        
        # Récupérer les scores de genre (déjà sous forme de dictionnaire)
        gender_scores = emotions[0]['gender']
        
        # Calculer le genre dominant en utilisant les scores de genre
        if gender_scores['Man'] > gender_scores['Woman']:
            dominant_gender = 'Homme'
        else:
            dominant_gender = 'Femme'
        
        # Générer une clé unique pour le visage basée sur les informations de genre et d'émotion
        face_key = f"{dominant_gender}_{dominant_emotion}"
        
        # Vérifier si le visage existe déjà dans le dictionnaire
        if face_key in face_dict:
            # Mettre à jour la fréquence
            face_dict[face_key]['Frequency'] += 1
        else:
            # Ajouter le visage au dictionnaire avec les informations
            face_dict[face_key] = {
                'ID': len(face_dict) + 1,
                'Date': None,
                'Time': None,
                'Age': age,
                'Gender': dominant_gender,
                'Emotion': dominant_emotion,
                'Frequency': 1
            }
        
        # Récupérer la date et l'heure actuelles
        current_datetime = datetime.datetime.now()
        date = current_datetime.date()
        time = current_datetime.time()
        
        # Catégoriser l'âge
        age_category = categorize_age(age)
        
        # Mettre à jour les informations de date, d'heure et de catégorie d'âge dans le dictionnaire
        face_dict[face_key]['Date'] = date
        face_dict[face_key]['Time'] = time
        face_dict[face_key]['Age Category'] = age_category
        
        # Écrire les informations dans le fichier CSV
        for face_info in face_dict.values():
            csv_writer.writerow([face_info['ID'], face_info['Date'], face_info['Time'], face_info['Age'], face_info['Age Category'], face_info['Gender'], face_info['Emotion'], face_info['Frequency']])
        
        # Afficher l'image avec les annotations
        cv2.imshow('Emotion Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    csv_file.close()
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    detect_emotion_from_stream()


TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [37]:
import pandas as pd
dat=pd.read_csv('C:/Users/lenovo/Desktop/tesst/deepface/facial_information/emotions.csv')
dat.head()

ID        Date             Time  Age  Age Category Gender  Emotion  \
0   1  2023-08-14  20:43:57.907102   35  young_adults  Homme    angry   
1   1  2023-08-14  20:43:57.907102   35  young_adults  Homme    angry   
2   2  2023-08-14  20:43:59.674920   32  young_adults  Homme  neutral   

   Frequency  
0          1  
1          1  
2          1

In [40]:
dataaa=pd.read_csv('resultats.csv')
dataaa.head()

id                                               name  \
0  6Pkt6qVikqPBt9bEQy8iTz                                 A Lover's Concerto   
1  1hx7X9cMXHWJjknb9O6Ava  The September Of My Years - Live At The Sands ...   
2  19oquvXf3bc65GSqtPYA5S  It Was A Very Good Year - Live At The Sands Ho...   
3  67BBzbF6t4ejG7HwVjQ4Tm  Love For Sale - Live At Chez Club, Hollywood/1...   
4  2oNV9ABtLyb0SvhXZibgL0  You've Lost That Lovin' Feelin' - 2006 Digital...   

   danceability  energy  valence  year year_range  plage_id  
0         0.671   0.867    0.839  2020  2009-2021         5  
1         0.319   0.201    0.239  2018  2009-2021         5  
2         0.269   0.129    0.160  2018  2009-2021         5  
3         0.409   0.839    0.557  1997  1986-2001         3  
4         0.400   0.335    0.311  2006  2002-2008         4

In [43]:
# Lire les données d'émotion depuis le fichier emotion.csv
emotion_data = pd.read_csv('C:/Users/lenovo/Desktop/tesst/deepface/facial_information/emotions.csv')

# Obtenir les informations d'âge et de genre dominants depuis le fichier emotion.csv
dominant_age = emotion_data['Age Category'].mode().iloc[0]
dominant_gender = emotion_data['Gender'].mode().iloc[0]

# Lire les données musicales depuis le fichier resultats.csv
music_data = pd.read_csv('resultats.csv')

# Définir les plages d'âge en fonction du dominant_age
if dominant_age == 'children':
    year_range = [2009, 2021]
elif dominant_age == 'teenagers':
    year_range = [2002, 2008]
elif dominant_age == 'young_adults':
    year_range = [1986, 2001]
elif dominant_age == 'middle_adults':
    year_range = [1971, 1985]
elif dominant_age == 'elderly':
    year_range = [-float('inf'), 1970]

# Filtrer les chansons en fonction de la plage d'année
filtered_music = music_data[(music_data['year'] >= year_range[0]) & (music_data['year'] <= year_range[1])]

# Filtrer davantage en fonction du genre dominant
if dominant_gender == 'Homme':
    filtered_music = filtered_music[(filtered_music['valence'] > 0) & (filtered_music['valence'] < 0.5) &
                                    (filtered_music['danceability'] > 0.7) & (filtered_music['danceability'] < 1) &
                                    (filtered_music['energy'] > 0.7) & (filtered_music['energy'] < 1)]
elif dominant_gender == 'Femme':
    filtered_music = filtered_music[(filtered_music['valence'] > 0.5) & (filtered_music['valence'] < 1) &
                                    (filtered_music['danceability'] > 0.5) & (filtered_music['danceability'] < 1) &
                                    (filtered_music['energy'] > 0.5) & (filtered_music['energy'] < 1)]

# Save the filtered music data to a new CSV file
filtered_music.to_csv('C:/Users/lenovo/Desktop/tesst/deepface/playliste/playliset1.csv', index=False)

In [44]:
dt=pd.read_csv('C:/Users/lenovo/Desktop/tesst/deepface/playliste/playliset1.csv')

# Get the number of elements (rows) in the DataFrame
num_elements = len(dt)

# Print the number of elements
print("Les musiques sélectionnées sont au nombre de :", num_elements)

dt.head()

Les musiques sélectionnées sont au nombre de : 646


id                                               name  \
0  2WBzF0a3DDzqjNF8V3ApAw  That Was Your Mother (with Good Rockin' Doopsi...   
1  4Kvz5bhHNYPSaGnQ7PsTBi                                    Change of Heart   
2  5ph3PnU1q2x4rcp94aiBiA                  Mean Green Mother From Outerspace   
3  5lWFrW5T3JtxVCLDb7etPu                                Bad - 2012 Remaster   
4  1kiNatIrwDusOZfR29W0LJ                  Man in the Mirror - 2012 Remaster   

   danceability  energy  valence  year year_range  plage_id  
0         0.722   0.757    0.483  1986  1971-1985         3  
1         0.706   0.847    0.495  1986  1971-1985         3  
2         0.715   0.853    0.407  1986  1971-1985         3  
3         0.787   0.889    0.394  1987  1986-2001         3  
4         0.808   0.812    0.269  1987  1986-2001         3

In [45]:
import pandas as pd
import requests

# Charger la playlist filtrée à partir du fichier CSV
filtered_music = pd.read_csv('C:/Users/lenovo/Desktop/tesst/deepface/playliste/playliset1.csv')

# Liste pour stocker les informations sur les musiques sélectionnées
selected_music_info = []

# Parcourir chaque musique dans la playlist filtrée
for index, row in filtered_music.iterrows():
    # Récupérer le titre de la musique
    title = row['name']  # Utiliser le nom de colonne correct pour le titre
    carac1=row['danceability']
    carac2=row['energy']
    carac3=row['valence']
    
    # Rechercher la musique sur l'API iTunes Search en utilisant le titre
    url = f'https://itunes.apple.com/search?term={title}&entity=song&limit=1'
    response = requests.get(url)
    
    # Vérifier si la requête a réussi
    if response.status_code == 200:
        data = response.json()
        # Vérifier si des résultats ont été trouvés
        if data['resultCount'] > 0:
            # Prendre la première musique dans les résultats
            first_result = data['results'][0]
            
            # Récupérer l'ID de la musique et le lien Apple Music
            music_id = first_result['trackId']
            apple_music_link = first_result['trackViewUrl']
            
            # Ajouter les détails de la musique à la liste
            selected_music_info.append({
                'title': title,
                'apple_music_id': music_id,
                'apple_music_link': apple_music_link,
                'valence': carac3,
                'Energie': carac1,
                'Danceability':carac2
            })
        else:
            # Si aucune correspondance n'a été trouvée, ajouter une entrée avec des informations manquantes
            selected_music_info.append({
                'title': title,
                'apple_music_id': None,
                'apple_music_link': None,
                'valence': None,
                'Energie': None,
                'Danceability':None
            })
    
# Créer un DataFrame avec les informations des musiques sélectionnées
selected_music_df = pd.DataFrame(selected_music_info)

# Enregistrer les informations des musiques sélectionnées dans un fichier CSV
selected_music_df.to_csv('C:/Users/lenovo/Desktop/tesst/deepface/lien_musique/selected_music_with_apple_music.csv', index=False)

# Afficher les informations des musiques sélectionnées
print(selected_music_df)


                                                 title  apple_music_id  \
0    That Was Your Mother (with Good Rockin' Doopsi...             NaN   
1                                      Change of Heart    1.874888e+08   
2                    Mean Green Mother From Outerspace    1.434904e+09   
3                                  Bad - 2012 Remaster    1.473415e+09   
4                    Man in the Mirror - 2012 Remaster             NaN   
..                                                 ...             ...   
221         The Gangsta, The Killa And The Dope Dealer    7.246908e+08   
222                                        Jungle Bill    1.443864e+09   
223                  Boing Boom Tschak - 2009 Remaster             NaN   
224                  Everyday Struggle - 2005 Remaster             NaN   
225                                               Flim    1.935367e+08   

                                      apple_music_link  valence  Energie  \
0                                  

In [46]:
datt=pd.read_csv('C:/Users/lenovo/Desktop/tesst/deepface/lien_musique/selected_music_with_apple_music.csv')
datt.head()

title  apple_music_id  \
0  That Was Your Mother (with Good Rockin' Doopsi...             NaN   
1                                    Change of Heart    1.874888e+08   
2                  Mean Green Mother From Outerspace    1.434904e+09   
3                                Bad - 2012 Remaster    1.473415e+09   
4                  Man in the Mirror - 2012 Remaster             NaN   

                                    apple_music_link  valence  Energie  \
0                                                NaN      NaN      NaN   
1  https://music.apple.com/us/album/change-of-hea...    0.495    0.706   
2  https://music.apple.com/us/album/mean-green-mo...    0.407    0.715   
3  https://music.apple.com/us/album/one-bad-habit...    0.394    0.787   
4                                                NaN      NaN      NaN   

   Danceability  
0           NaN  
1         0.847  
2         0.853  
3         0.889  
4           NaN

In [60]:
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
import pandas as pd
import datetime

# Charger les données à partir des fichiers CSV
emotion_data = pd.read_csv('C:/Users/lenovo/Desktop/tesst/deepface/facial_information/emotions.csv')
music_data = pd.read_csv('resultats.csv')

# Calculer la moyenne des fréquences de visite
average_frequency = emotion_data['Frequency'].mean()

# Obtenir la liste des émotions détectées
emotions_detected = emotion_data['Emotion'].unique()

# Calculer les genres détectés avec leurs nombres
gender_counts = emotion_data['Gender'].value_counts()

# Obtenir le genre dominant
dominant_gender = gender_counts.idxmax()

# Obtenir l'âge dominant
dominant_age = emotion_data['Age'].mode().iloc[0]

# Obtenir l'émotion dominante
dominant_emotion = emotion_data['Emotion'].mode().iloc[0]

# Obtenir l'heure de pointe
# Convertir la colonne "Time" en format datetime
emotion_data['Time'] = pd.to_datetime(emotion_data['Time'])  # Utilisez 'Time' ici

# Extraire l'heure à partir de la colonne "Time"
emotion_data['hour'] = emotion_data['Time'].dt.hour

# Regrouper les données par heure et compter le nombre de lignes dans chaque groupe
hourly_counts = emotion_data.groupby('hour').size()

# Trouver l'heure de pointe (heure avec le nombre maximal de lignes)
peak_hour = hourly_counts.idxmax()


# Filtrer et récupérer les informations sur les musiques sélectionnées depuis le fichier playliste.csv
filtered_music = pd.read_csv('C:/Users/lenovo/Desktop/tesst/deepface/playliste/playliset1.csv')
selected_music_count = len(filtered_music)

# Créer un document PDF
pdf_filename = 'C:/Users/lenovo/Desktop/tesst/deepface/rapport/music_analysis_report.pdf'
doc = SimpleDocTemplate(pdf_filename, pagesize=letter)

# Créer une liste pour contenir les éléments à ajouter au document
elements = []

# Ajouter la date de génération du rapport
now = datetime.datetime.now()
date_generated = now.strftime("%Y-%m-%d %H:%M:%S")
date_text = f"Rapport généré le {date_generated}"
date_paragraph = Paragraph(date_text, getSampleStyleSheet()['Normal'])
elements.append(date_paragraph)

# Ajouter un espace
elements.append(Paragraph("<br/>", getSampleStyleSheet()['Normal']))

# Ajouter un titre
title_style = getSampleStyleSheet()['Title']
title = Paragraph("Rapport d'Analyse Musicale", title_style)
elements.append(title)

# Ajouter un espace
elements.append(Paragraph("<br/>", getSampleStyleSheet()['Normal']))



# Ajouter les informations au document
report_info = [
    ["Tranches d'âge", "Moyenne des fréquences de visite", "Émotions détectées"],
    [dominant_age, round(average_frequency, 2), ', '.join(emotions_detected)],
    ["Genres détectés", "Genre dominant", "Émotion dominante"],
    [', '.join(f"{gender}: {count}" for gender, count in gender_counts.items()), dominant_gender, dominant_emotion],
    ["Heure de pointe", "Musiques sélectionnées et leur nombre", ""],
    [peak_hour, f"{selected_music_count} musiques", ""]
]

# Créer un tableau à partir des informations
table = Table(report_info, colWidths=[150, 150, 150])
table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.orange),
    ('TEXTCOLOR', (0, 0), (-1, -1), colors.white),  # Changement ici pour tout le texte
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
    ('BACKGROUND', (0, 1), (-1, -1), colors.black),
    ('GRID', (0, 0), (-1, -1), 1, colors.white),
]))

elements.append(table)

# Construire le document PDF
doc.build(elements)
print(f"Le document PDF {pdf_filename} a été créé avec succès.")

Le document PDF C:/Users/lenovo/Desktop/tesst/deepface/rapport/music_analysis_report.pdf a été créé avec succès.
